In [ ]:
%reset
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import matplotlib.pyplot as plt

import DeformationModules as defmod
import Shooting as shoot
import Hamiltonian as ham
import UsefulFunctions as fun
import Shooting

In [ ]:
d = 2

In [ ]:
sigma = 0.5
nbPtsTrans = 10
nbPtsSilent = 25

Trans = defmod.Translations(sigma, d, nbPtsTrans)
Trans2 = defmod.Translations(sigma, d, nbPtsTrans2)
Silent = defmod.SilentPoints(d, nbPtsSilent)

Compound = defmod.Compound([Trans, Silent])

In [ ]:
Hamiltonian = ham.Hamilt(Compound)

def H_r(GD, MOM):
    return Hamiltonian(GD, MOM, Hamiltonian.Cont_geo(GD, MOM))

In [ ]:
GD_Trans = torch.rand(nbPtsTrans, d, requires_grad=True).view(-1)
GD_Trans2 = torch.rand(nbPtsTrans2, d, requires_grad=True).view(-1)
GD_Silent = torch.rand(nbPtsSilent, d, requires_grad=True).view(-1)

MOM_Trans = torch.rand(nbPtsTrans, d, requires_grad=True).view(-1)
MOM_Trans2 = torch.rand(nbPtsTrans2, d, requires_grad=True).view(-1)
MOM_Silent = torch.rand(nbPtsSilent, d, requires_grad=True).view(-1)

GD_Comp = torch.cat([GD_Trans, GD_Silent])
MOM_Comp = torch.cat([MOM_Trans, MOM_Silent])

In [ ]:
Controls = Hamiltonian.Cont_geo(GD_Comp, MOM_Comp).view(-1)

In [ ]:
GD_Final, MOM_Final = Shooting.shoot(Compound, GD_Comp, MOM_Comp, H_r, 100)

In [ ]:
nx, ny = 100, 100
sx, sy = 10, 10
x, y = torch.meshgrid([torch.arange(0, nx), torch.arange(0, ny)])
x = sx*(x.type(torch.FloatTensor)/nx - 0.5)
y = sy*(y.type(torch.FloatTensor)/ny - 0.5)
u, v = fun.vec2grid(Compound(GD_Final, Controls, fun.grid2vec(x, y).type(torch.FloatTensor)), nx, ny)

plt.quiver(x.numpy(), y.numpy(), u.detach().numpy(), v.detach().numpy())
plt.show()